In [80]:
import socks
import socket
from bs4 import BeautifulSoup

def create_connection(address, timeout=None, source_address=None):
    sock = socks.socksocket()
    sock.connect(address)
    return sock

socks.setdefaultproxy(socks.PROXY_TYPE_SOCKS5, "127.0.0.1", 9050)

# patch the socket module
socket.socket = socks.socksocket
socket.create_connection = create_connection

import urllib2
import requests
import pandas as pd

In [81]:
response = urllib2.urlopen('http://rrcc5uuudhh4oz3c.onion/?cmd=category&id=12')
soup = BeautifulSoup(response.read(), 'lxml')
threads = soup.find("table", {"class": "contentbox"})

In [82]:
thread_links = []
for thread in list(threads)[1:]:
    topic = list(thread.children)[0]
    link = list(topic.children)[0]['href']
    thread_links.append(link)

In [87]:
posts = []
for link in thread_links:
    response = urllib2.urlopen('http://rrcc5uuudhh4oz3c.onion'+link)
    soup = BeautifulSoup(response.read(), 'lxml')
    comments = soup.find("table", {"class": "contentbox"})
    
    for comment in list(comments)[1:]:
        user = list(comment.children)[0].text
        timestamp = list(list(comment.children)[1].children)[0].text
        content = list(comment.children)[1]
        for div in content.find_all("div", {'class':'right'}): 
            div.decompose()

        posts.append({'user':user, 'content ': content.text, 'timestamp': timestamp})

In [88]:
posts_df = pd.DataFrame(posts)

In [90]:
posts_df.head()

,content,timestamp,user
0,So I wanted to get into this website and they ...,14 weeks and 4 days ago,Jappa
1,whats the site ? i can do a vulnerability scan...,14 weeks and 4 days ago,deadsh0t
2,a WpScan just identifies possible vulnerabilit...,14 hours and 50 minutes ago,digitalsurgeon
3,"hi guys,soon everyone from my class will need ...",4 days and 10 hours ago,ltribute
4,This thread is intended for people to discuss ...,1 week and 9 hours ago,PuppetMaster


In [92]:
posts_df.to_csv("Intel_Exchange_Posts.csv", encoding='utf-8')